# Observing Blocks

In [1]:
from keckODL.target import Target, TargetList
from keckODL.kcwi import KCWIblueDetectorConfig, KCWIredDetectorConfig
from keckODL.kcwi import KCWIConfig
from keckODL.offset import Stare, StarSkyStar
from keckODL.block import ObservingBlock, ObservingBlockList

from astropy import units as u

# KCWI + KCRM Example

In [2]:
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5,
                 objecttype='science',
                 acquisition='blind')

In [3]:
kcwib_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
kcwir_1800s = KCWIredDetectorConfig(exptime=1800, binning='2x2')
kcwib_1800s.write('detector_KCWIb_1800s.txt')
kcwir_1800s.write('detector_KCWIr_1800s.txt')

In [4]:
kcwib_1800s.estimate_clock_time()

1906

In [5]:
kcwir_1800s.match_time(1906)

In [ ]:
med_slicer_b4800_r6500 = KCWIConfig(slicer='medium', bluegrating='BH3', bluecwave=4800,
                                    redgrating='RH3', redcwave=6563,)
med_slicer_b4800_r6500.write('KCWIconfig_medslicer_red_blue.txt')
med_slicer_b4800_r6500

In [ ]:
skyoffset = StarSkyStar(dx=60*u.arcsec, dy=30*u.arcsec)
skyoffset.write('offset_customStarSkyStar.txt')

In [ ]:
OB1 = ObservingBlock(target=ngc1333, pattern=skyoffset,
                            detconfig=(kcwib_1800s, kcwir_1800s),
                            instconfig=med_slicer_b4800_r6500,
                            repeat=3)

In [ ]:
OBlist = ObservingBlockList([OB1])
OBlist

In [ ]:
cals_I_need = OBlist.cals()
cals_I_need

In [ ]:
cals_I_need.estimate_time()

## MOSFIRE Example

In [ ]:
from keckODL.target import Target, TargetList
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList

In [ ]:
v1647Ori = Target('V1647 Ori', rotmode='PA', PA=22.5,
                  objecttype='science',
                  acquisition='blind')

In [ ]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
print(mosfire_180s)
print(mosfire_120s)

In [ ]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

In [ ]:
abba = ABBA(offset=1.5)
print(abba)

In [ ]:
abba

In [ ]:
for i,offset in enumerate(abba):
    print(f'Executing offset: {i}')
    offset.execute()

In [ ]:
myOBs = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                           ])
print(myOBs)

In [ ]:
myOBs.estimate_time()

In [ ]:
cals_I_need = mosfire_Y.cals()
cals_I_need.extend( mosfire_J.cals() )
cals_I_need.extend( mosfire_H.cals() )
cals_I_need.extend( mosfire_K.cals() )
print(cals_I_need)

In [ ]:
cals_I_need.estimate_time()

## MOSFIRE Example 2

In [ ]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

In [ ]:
long2pos_pattern = long2pos()
print(long2pos_pattern)

In [ ]:
long2pos_pattern

In [ ]:
long2pos_pattern[0].execute()

In [ ]:
myOBs2 = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                            ])
print(myOBs2)

In [ ]:
myOBs2.estimate_time()

In [ ]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

In [ ]:
cals_I_need.estimate_time()

# MOSFIRE Night Example

In [ ]:
myOBs3 = ObservingBlockList([ObservingBlock(target=ngc1333, pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                            ])
print(myOBs3)

In [ ]:
myOBs.extend(myOBs2)
myOBs.extend(myOBs3)

In [ ]:
myOBs

In [ ]:
myOBs.cals()